In [2]:
import requests
from docx import Document
import uuid


subscription_key = "YOUR_SUBSCRIPTION_KEY"
endpoint = "YOUR_ENDPOINT"
location = "YOUR_LOCATION"

target_language = "pt-br"


def translator_document(text, target_language):
    path = "/translate?api-version=3.0"
    constructed_url = endpoint + path
    headers = {
        "Ocp-Apim-Subscription-Key": subscription_key,
        "Ocp-Apim-Subscription-Region": location,
        "Content-type": "application/json",
        "X-ClientTraceId": str(uuid.uuid4())
    }

    body = [{
        "text": text
    }]

    params = {
        "api": "translate",
        "from": "en",
        "to": [target_language]
    }
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    return request.json()[0]['translations'][0]['text']


In [ ]:
text = "Your text here"
translated_text = translator_document(text, target_language)
print(translated_text)


In [ ]:
def translate_document(document_path, target_language):
    doc = Document(document_path)
    full_text = []
    for paragraph in doc.paragraphs:
        translated_text = translator_document(paragraph.text, target_language)
        full_text.append(translated_text)
        
    translated_doc = Document()    
    for line in full_text:
        translated_doc.add_paragraph(line)

    path = document_path.split('.')[0] + '_translated.docx'
    translated_doc.save(document_path)

In [ ]:
input_path = 'input.docx'
output_path = 'output.docx'
translate_document(input_path, target_language)

In [ ]:
import requests
from bs4 import BeautifulSoup4

def extract_text_from_html(html_path):
    response = requests.get(html_path)
    
    if response.status_code == 200:
        soup = BeautifulSoup4(response.text, 'html.parser')

        for script_or_style in soup(['script', 'style']):
            script_or_style.decompose()
            
        text = soup.get_text(separator=' ', strip=True)
        #Clean text
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split('  '))
        clean_text = '\n'.join(chunk for chunk in chunks if chunk)
        return clean_text

    else:
        raise Exception(f"Failed to retrieve the HTML file: {response.status_code}")

html_path = 'input.html'
extracted_text = extract_text_from_html(html_path)

In [ ]:
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain_openai.embeddings.azure import AzureOpenAIEmbeddings

client = AzureChatOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    azure_deployment="gpt-4o",
    api_version="2024-02-15-preview",
    max_retries=3,
    temperature=0.0
)

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    azure_deployment="text-embedding-3-large"
)

def translate_article(text, target_language):
    messages =[
        ("system", "Você atua como tradutor de textos técnicos. Você recebe um texto e traduz para o português brasileiro."),
        ("user", f"Traduz o seguinte texto para o português brasileiro: {text}")
    ]

    response = client.invoke(messages)
    return response.content[0].text

In [ ]:
url = "URL_DO_ARTIGO_EM_INGLES"
extracted_text = extract_text_from_html(url)
translated_text = translate_article(extracted_text, target_language)
print(translated_text)